In [108]:
import pandas as pd
import numpy as np
import math
from sklearn.tree import export_graphviz, DecisionTreeClassifier

def factorize_df(df):
    for x in list(df.columns.values):
        df[x] = pd.factorize(df[x])[0]
        
def to_df(dic, feature_list):
    df = pd.DataFrame(dic)
    df = pd.concat([df, pd.get_dummies(df[feature_list])], axis = 1)
    df.drop(feature_list, axis = 1, inplace = True)
    return df

features = ['Внешность', 'Алкоголь','Красноречие', 
            'Потраченные деньги']

data_bar_x = {}
data_bar_y = {}
data_bar_test = {}
data_bar_x['Внешность'] = ['Приятный', 'Приятный', 'Приятный', 'Неприятный', 
                               'Неприятный', 'Неприятный', 'Приятный']
data_bar_x['Алкоголь'] = ['да', 'да', 'нет', 'нет', 'да', 
                               'да', 'да']
data_bar_x['Красноречие'] = ['высокий', 'низкий', 'средний', 'средний', 
                                 'низкий','высокий', 'средний']
data_bar_x['Потраченные деньги'] = ['много', 'мало', 'много', 'мало', 
                                 'много','много', 'много']

data_bar_y['Поедет'] = ['+', '-', '+', '-', '-', '+', '+']

data_bar_test['Внешность'] = ['Приятный', 'Приятный', 'Неприятный']
data_bar_test['Алкоголь'] = ['нет', 'да','да']
data_bar_test['Красноречие'] = ['средний','высокий', 'средний']
data_bar_test['Потраченные деньги'] = ['много', 'мало', 'много']

data_bar_x = to_df(data_bar_x, features)
data_bar_y = pd.DataFrame(data_bar_y)
data_bar_test = to_df(data_bar_test, features)

factorize_df(data_bar_y)
    
tree =  DecisionTreeClassifier(criterion = 'entropy')
tree.fit(data_bar_x, data_bar_y)
export_graphviz(tree,feature_names = data_bar_x.columns, out_file='tree.dot', filled=True )

tree_app = DecisionTreeClassifier(criterion = 'entropy')
tree_app.fit(data_bar_x[['Внешность_Приятный']], data_bar_y)
export_graphviz(tree_app,feature_names = ['Приятный'], out_file='tree2.dot', filled=True )

In [109]:
def entropy(list):
    p = dict()
    for i in set(list):
        p[i] = list.count(i)
    return sum([-(x/len(list))*(math.log(x/len(list)))/math.log(2) for x in p.values()])

balls = [1 for i in range(9)] + [0 for i in range(11)]
balls_left  = [1 for i in range(8)] + [0 for i in range(5)] 
balls_right = [1 for i in range(1)] + [0 for i in range(6)] 

print(entropy(balls))
print(entropy(balls_left)) 
print(entropy(balls_right))
print(entropy([1,2,3,4,5,6]))

0.9927744539878083
0.961236604722876
0.5916727785823274
2.584962500721156


In [125]:
from sklearn.metrics import accuracy_score

train = pd.read_csv('Data/adult_train.csv', sep = ';')
test = pd.read_csv('Data/adult_test.csv', sep = ';')

test = test[(test['Target'] == ' >50K.') | (test['Target'] ==' <=50K.')]

test.at[test['Target'] == ' <=50K.', 'Target' ] = 0
test.at[test['Target'] == ' >50K.', 'Target' ] = 1

train.at[train['Target'] == '<=50K', 'Target' ] = 0
train.at[train['Target'] == '>50K', 'Target' ] = 1

test['Age'] =  test['Age'].astype(int)
test['fnlwgt'] = test['fnlwgt'].astype(int)
test['Education_Num'] = test['Education_Num'].astype(int)
test['Capital_Gain'] = test['Capital_Gain'].astype(int)
test['Capital_Loss'] = test['Capital_Loss'].astype(int)
test['Hours_per_week'] = test['Hours_per_week'].astype(int)


categorical_columns_train = [c for c in train.columns 
                             if train[c].dtype.name == 'object']
numerical_columns_train = [c for c in train.columns 
                           if train[c].dtype.name != 'object']

categorical_columns_test = [c for c in test.columns 
                            if test[c].dtype.name == 'object']
numerical_columns_test = [c for c in test.columns 
                          if test[c].dtype.name != 'object']


for c in categorical_columns_train:
    train[c] = train[c].fillna(train[c].mode())
for c in categorical_columns_test:
    test[c] = test[c].fillna(test[c].mode())
    
for c in numerical_columns_train:
    train[c] = train[c].fillna(train[c].median())
for c in numerical_columns_test:
    test[c] = test[c].fillna(test[c].median())


columns = [ 'Workclass', 'Education' , 'Martial_Status', 'Occupation', 'Relationship',
          'Race', 'Sex', 'Country']

train = pd.concat([train,pd.get_dummies(train[columns])],axis=1)
test = pd.concat([test,pd.get_dummies(test[columns])],axis=1)

train.drop(columns, axis=1, inplace=True)
test.drop(columns, axis=1, inplace=True)

test['Country_ Holand-Netherlands'] = np.zeros([test.shape[0], 1])

tree_adult =  DecisionTreeClassifier( max_depth = 9
                        , random_state = 17)

train_y = train['Target']
test_y = test['Target']
train.drop(['Target'], axis=1, inplace=True)
test.drop(['Target'], axis=1, inplace=True)
tree_adult.fit(train, train_y)
tree_predictions = tree_adult.predict(test)
accuracy_score(test_y, tree_predictions)

0.847122412628217